In [1]:
import warnings

warnings.filterwarnings('ignore', message='Unverified HTTPS request')

class Session():
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port

session = Session("127.0.0.1", "8000")

# 1. Aggregation Model Configuration

In [2]:
from smart_broker_api import data_model_api


data_model_id = data_model_api.create_date_frame(session, "data_frame_1")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_mean", "Observation:Body Mass Index", "AgregatorIntervalMean")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_first", "Observation:Body Mass Index", "AgregatorIntervalFirstOccurance")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_last", "Observation:Body Mass Index", "AgregatorIntervalLastOccurance")
data_model_tabular_id = data_model_api.create_tabular_data(session)
data_model_tabular_id = data_model_api.tabular_add_dataframe(session, data_model_id, data_model_tabular_id)

# 2. Flatten / Aggregate data from Aggregation model

In [4]:
from smart_broker_api import data_api

dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
dataset_federation_name = "r4sep2019_csvv1_20_1"

longitudinal_id = data_api.read_longitudinal_fhirv1(session)
tabular_dataset_id = data_api.parse_dataset_tabular_from_longitudinal(session, longitudinal_id, dataset_federation_id, dataset_federation_name, data_model_tabular_id)
data_frame_id = data_api.data_frame_tabular_select_data_frame(session, tabular_dataset_id, "data_frame_1")

# 3. Remove NaNs from data

In [5]:
from smart_broker_api import preprocessing_api

no_nan_data_frame_id = preprocessing_api.drop_na_data_frame(session, data_frame_id)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# 4. Grab series to work with in Statistics Lib

In [ ]:
series_1_id = data_api.data_frame_select_series(session, no_nan_data_frame_id, "bmi_mean")
series_2_id = data_api.data_frame_select_series(session, no_nan_data_frame_id, "bmi_last")

print(series_1_id+" , "+series_2_id)

# 4. Query a data frame Cohort / Feature Selection

In [ ]:
from smart_broker_api import preprocessing_api
from smart_broker_api import statistics_api

query = "bmi_first > bmi_last"
query_dataframe = preprocessing_api.query_data_frame(session, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(session, query_dataframe, "bmi_last")

print(f"Patients whose BMI decreased during study: {statistics_api.count(session,  query_series)}")

query = "bmi_first < bmi_last"
query_dataframe = preprocessing_api.query_data_frame(session, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(session, query_dataframe, "bmi_last")

print(f"Patients whose BMI increased during study: {statistics_api.count(session,  query_series)}")

query = "bmi_first - bmi_last > 0.5"
query_dataframe = preprocessing_api.query_data_frame(session, no_nan_data_frame_id, query)
query_series = data_api.data_frame_select_series(session, query_dataframe, "bmi_last")

print(f"Patients whose BMI significantly increased during study: {statistics_api.count(session,  query_series)}")

print(f"Mean BMI of patients whose BMI increased: {statistics_api.mean(session, query_series)} ")

# 5. Run Statistics Library

In [ ]:
from smart_broker_api import statistics_api

type_distribution="normalunit" 
type_ranking="cdf"
alternative = "two-sided"
print(statistics_api.count(session,  series_1_id))
print(statistics_api.mean(session,  series_1_id))
print(statistics_api.chisquare(session,  series_1_id, series_2_id))
print(statistics_api.kolmogorovSmirnovTest(session,  series_1_id, type_distribution, type_ranking))
print(statistics_api.kurtosis(session,  series_1_id))
print(statistics_api.levene_test(session,  series_1_id, series_2_id))
print(statistics_api.mann_whitney_u_test(session,  series_1_id, series_2_id, alternative, type_ranking))
print(statistics_api.min_max(session,  series_1_id))
print(statistics_api.paired_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.pearson(session,  series_1_id, series_2_id, alternative))
print(statistics_api.skewness(session,  series_1_id))
print(statistics_api.spearman(session,  series_1_id, series_2_id, alternative, type_ranking)) 
print(statistics_api.student_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.variance(session,  series_1_id))
print(statistics_api.welch_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.wilcoxon_signed_rank_test(session,  series_1_id, series_2_id, alternative, type_ranking))





